# **Funzioni utili**
Definiamo alcune funzioni per la suddivisione ed il calcolo di alcune statistiche dei dataset


In [32]:
import os
import torch
import shutil
from PIL import Image
import xml.etree.ElementTree as ET
from torchvision.transforms.functional import pil_to_tensor, resize
from torch.nn.functional import pad
import random
import pandas as pd
import numpy as np
from itertools import islice
from sklearn import preprocessing
%run path.ipynb


### **1. Statistiche IAM**
Questa funzione scorre tutto il dataset IAM e calcola le dimensioni massime raggiunte dalle immagini.

In [33]:
def get_IAM_statistics():
    h = []
    w = []
    # images = []
    main_dir = IAM + '/sentences'
    for dir in os.listdir(main_dir):
        dir_path = os.path.join(main_dir, dir)
        for subdir in os.listdir(dir_path):
            subdir_path = os.path.join(dir_path, subdir)
            for image in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, image)
                img = Image.open(img_path).convert('RGB')
                # images.append(resize(pil_to_tensor(img) / 255., (128, 1024)))
                img_size = img.size
                h.append(img_size[1])
                w.append(img_size[0])
    # images = torch.stack(images, dim=0)
    # mean = torch.mean(images)
    # std = torch.std(images)

    # print(mean, std)
    
    return max(w), max(h)

### **2. Statistiche Dysgraphia**
Questa funzione scorre tutto il dataset Dysgraphia e calcola le dimensioni massime raggiunte dalle immagini.

In [34]:
def get_base_statistics(aug : bool = False):
    h = []
    w = []
    
    if aug == True:
        main_dir = ADYSG
    else:
        main_dir = DYSG
    
    for dir in os.listdir(main_dir):
        dir_path = os.path.join(main_dir, dir)
        for image in os.listdir(dir_path):
            img_path = os.path.join(dir_path, image)
            img = Image.open(img_path).convert('RGB')
            # images.append(resize(pil_to_tensor(img) / 255., (128, 1024)))
            img_size = img.size
            h.append(img_size[1])
            w.append(img_size[0])
    # images = torch.stack(images, dim=0)
    # mean = torch.mean(images)
    # std = torch.std(images)

    # print(mean, std)
    
    return max(w), max(h)

### **3. Suddivisione Dysgraphia**
Questa funzione suddivide il dataset etichettato Dysgraphia in train, test e validation set e li salva in file di testo, successivamente considerati dall'architettura.

In [35]:
def create_simple_splits(path : str):
    if os.path.isdir(os.path.join(path, 'train.txt')):
        return
    else:
        print("Creating Simple splits.")
    
    dis = [filename for filename in os.listdir(str(path) + "/Dysgraphic1") and os.listdir(str(path) + "/Dysgraphic2")]
    not_dis = [filename for filename in os.listdir(str(path) + "/No_Dysgraphic1") and os.listdir(str(path) + "/No_Dysgraphic2")]

    print(len(dis))
    print(len(not_dis))

    test_dis = random.sample(dis, 3)
    validation = [random.choice(test_dis)]
    test_dis.remove(validation[0])

    test_not_dis = random.sample(not_dis, 3)
    validation.append(random.choice(test_not_dis))
    test_not_dis.remove(validation[1])

    test_dis.extend(test_not_dis)
    train = [filename for filename in os.listdir(path) if filename not in test_dis]

    with open(os.path.join('/'.join(str(path).split("/")[:-1]), 'train.txt'), 'w') as f:
        for t in train:
            f.write(f"{t}\n")
    
    with open(os.path.join('/'.join(str(path).split("/")[:-1]), 'validation.txt'), 'w') as f:
        for t in validation:
            f.write(f"{t}\n")

    with open(os.path.join('/'.join(str(path).split("/")[:-1]), 'test.txt'), 'w') as f:
        for t in test_dis:
            f.write(f"{t}\n")

# dataset_path = ADYSG
# create_simple_splits(dataset_path)